In [52]:
USE CuatroCuadras

# Cuatro Cuadras #
## Consultas##

## 1
Qué lugares son los más frecuentados por personas mayores a 45 años

In [53]:
SELECT TOP 3 count(*) AS visitas, L.ID_Lugar, L.Nombre FROM Lugar L
INNER JOIN Visita V  
ON L.ID_Lugar=V.ID_Lugar
INNER JOIN Usuario U 
ON V.Nickname=U.Nickname
WHERE DATEDIFF(YEAR,Fecha_Nacimiento,GETDATE())>45
GROUP BY L.ID_Lugar, L.Nombre

## 2
Cuáles son los dos lugares menos visitados

In [54]:
SELECT TOP 2 count(*) AS Visitas, L.ID_Lugar, L.Nombre FROM LUGAR L 
INNER JOIN VISITA V 
ON L.ID_Lugar=V.ID_Lugar
GROUP BY L.ID_Lugar, L.Nombre
ORDER BY Visitas ASC

## 3
Cuáles son los lugares que tienen "muy malo" o "malo" como comentario y ningun "Muy Bueno", "bueno" o "excelente"

In [55]:
SELECT DISTINCT L.ID_Lugar, L.Nombre FROM Lugar L
INNER JOIN VISITA V 
ON L.ID_Lugar=V.ID_Lugar
WHERE v.Comentario LIKE '%malo%' AND V.ID_Lugar NOT IN 
(SELECT DISTINCT ID_Lugar FROM Visita WHERE Comentario LIKE '%bueno%' OR Comentario LIKE '%excelente%' OR Comentario LIKE '%Muy bueno%')

## 4
Cuáles son los 3 lugares qué cuentan con la mejor valoración

In [56]:
WITH Promedio (ID_Lugar, Nombre, Valoracion)
AS
(
    SELECT l.ID_Lugar, L.Nombre, AVG(V.Valoracion) AS ValoracionTotal FROM LUGAR L
    INNER JOIN VISITA V 
    ON L.ID_Lugar=V.ID_Lugar
    GROUP BY L.ID_Lugar, L.Nombre
)
SELECT TOP 3 * FROM Promedio
ORDER BY Valoracion DESC

## 5
Qué usuarios han obtenido cuando menos un logro

In [57]:
SELECT Nickname, CONCAT(Nombre, ' ', Apellidos) as Nombre FROM USUARIO U 
WHERE U.Nickname IN (SELECT Nickname FROM LOGRO_USUARIO)

## 6
Cuáles han sido los usuarios que han hecho visitas en una ciudad distinta a la de su origen

In [58]:
SELECT DISTINCT U.Nickname AS Usuario FROM USUARIO U 
INNER JOIN VISITA V 
ON U.Nickname=V.Nickname
INNER JOIN LUGAR L 
ON V.ID_Lugar=L.ID_Lugar
WHERE U.ID_Ciudad<>L.ID_Ciudad

## 7
Qué usuarios han hecho amistad con usuarios de una ciudad distinta a la de ellos

In [59]:
SELECT A.Nickname1 AS [Usuario 1], A.Nickname2 AS [Usuario 2] FROM AMIGO A 
INNER JOIN USUARIO U 
ON A.Nickname1=U.Nickname
INNER JOIN USUARIO U2 
ON A.Nickname2=U2.Nickname
WHERE U.ID_Ciudad<>U2.ID_Ciudad

## 8
Se requiere saber el lugar más frecuentado por mujeres

In [60]:
WITH Frecuencia(Nombre, Visitas )
AS 
(
    SELECT L.Nombre, count(*) AS Visitas FROM LUGAR L 
    INNER JOIN VISITA V 
    ON L.ID_Lugar=V.ID_Lugar
    INNER JOIN USUARIO U 
    ON V.Nickname=U.Nickname
    WHERE U.Sexo='F'
    GROUP BY L.Nombre
)
SELECT TOP 1 * FROM Frecuencia
ORDER BY Visitas DESC

## 9
Conocer todos los usuarios que han comentado con "Muy malo" a sus visitas

In [61]:
SELECT U.Nickname FROM VISITA V 
INNER JOIN USUARIO U 
ON V.Nickname=U.Nickname 
WHERE V.Comentario='Muy malo'

## 10
Se desea saber cuál es el promedio de valoración de todos los lugares

In [62]:
SELECT L.Nombre,  AVG(V.Valoracion) AS Valoracion FROM LUGAR L 
INNER JOIN VISITA V 
ON L.ID_Lugar=V.ID_Lugar 
GROUP BY L.Nombre

## 11
Cuál es el lugar más visitado con la etiqueta de "Beisbol"

In [63]:
SELECT TOP 1 L.Nombre, COUNT(*) as Visitas FROM Lugar L
INNER JOIN Etiqueta E ON L.ID_Etiqueta=E.ID_Etiqueta
INNER JOIN Visita V on L.ID_Lugar=V.ID_Lugar
WHERE E.Nombre_eti='Beisbol'
GROUP BY L.Nombre

## 12
Cuál es el lugar menos visitado de Tijuana

In [64]:
SELECT TOP 1 L.Nombre, COUNT(*) as Visitas FROM Lugar L
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
INNER JOIN Ciudad C ON L.ID_Ciudad=C.ID_Ciudad
WHERE Ciudad='Tijuana'
GROUP BY L.Nombre
ORDER BY COUNT(*) ASC

## 13
Cuales son los 5 lugares más visitados del 2018 además de la ciudad que pertenecen

In [65]:
SELECT TOP 5 L.Nombre, C.Ciudad, COUNT(*) as Visitas FROM Lugar L
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
INNER JOIN Ciudad C ON L.ID_Ciudad=C.ID_Ciudad
WHERE YEAR(Fecha)=2018
GROUP BY L.Nombre, C.Ciudad
ORDER BY Visitas DESC

## 14
Qué ciudad tiene menos visitas registradas

In [66]:
SELECT TOP 1 C.Ciudad, COUNT(*) Visitas FROM Ciudad C
INNER JOIN Lugar L ON C.ID_Ciudad=L.ID_Ciudad
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
GROUP BY C.Ciudad
ORDER BY Visitas ASC

## 15
Los lugares más visitados de los últimos 3 meses

In [67]:
SELECT L.Nombre, COUNT(*) as Visitas,
CASE
    WHEN MONTH(Fecha)=1 THEN 'Enero'
    WHEN MONTH(Fecha)=2 THEN 'Febrero'
    WHEN MONTH(Fecha)=3 THEN 'Marzo'
    WHEN MONTH(Fecha)=4 THEN 'Abril'
    WHEN MONTH(Fecha)=5 THEN 'Mayo'
    WHEN MONTH(Fecha)=6 THEN 'Junio'
    WHEN MONTH(Fecha)=7 THEN 'Julio'
    WHEN MONTH(Fecha)=8 THEN 'Agosto'
    WHEN MONTH(Fecha)=9 THEN 'Septiembre'
    WHEN MONTH(Fecha)=10 THEN 'Octubre'
    WHEN MONTH(Fecha)=11 THEN 'Noviembre'
    WHEN MONTH(Fecha)=12 THEN 'Diciembre'
END AS Mes
FROM Visita V
INNER JOIN Lugar L ON V.ID_Lugar=L.ID_Lugar
WHERE Fecha BETWEEN DATEADD(MM, -3, GETDATE()) AND GETDATE()
GROUP BY L.Nombre, MONTH(Fecha)

## 16
Cuáles son los 5 mayores interéses de los usuarios

In [68]:
SELECT TOP 5 I.Interes, COUNT(*) AS Total FROM Interes_Usuario U
INNER JOIN Interes I ON U.ID_Interes=I.ID_Interes
GROUP BY I.Interes
ORDER BY Total DESC

## 17
Los usuarios que no han registrado visitas

In [69]:
SELECT U.Nickname FROM Usuario U
WHERE Nickname NOT IN (SELECT Nickname FROM Visita)

## 18
Los lugares que visita la persona de menor edad

In [70]:
DECLARE @Menor INT
SELECT @Menor=MIN(DATEDIFF(YEAR,Fecha_Nacimiento,GETDATE())) FROM Usuario

SELECT DISTINCT L.Nombre FROM Lugar L
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
INNER JOIN Usuario U ON V.Nickname=U.Nickname
WHERE DATEDIFF(YEAR,U.Fecha_Nacimiento,GETDATE())=@Menor

## 19
Obtener los 5 lugares más visitados por personas entre 18 y 23 años

In [71]:
SELECT TOP 5 COUNT(*) AS Visitas, L.Nombre FROM VISITA V
INNER JOIN USUARIO U ON V.Nickname = U.Nickname
INNER JOIN Lugar L ON L.ID_Lugar=V.ID_Lugar
WHERE DATEDIFF(YEAR,U.Fecha_Nacimiento,GETDATE())>=18 AND DATEDIFF(YEAR,U.Fecha_Nacimiento,GETDATE())<23
GROUP BY L.Nombre
ORDER BY Visitas DESC

## 20
Los lugares que tienen calificacion por debajo del promedio

In [72]:
Declare @promedio INT
Select @promedio=AVG(Valoracion) from Visita

SELECT L.Nombre, AVG(V.Valoracion) as Promedio FROM Lugar L
INNER JOIN Visita V ON V.ID_Lugar=L.ID_Lugar
GROUP BY L.Nombre
HAVING AVG(V.Valoracion)<@promedio
ORDER BY Promedio

## 21
Registros de las visitas con la mayor califación

In [73]:
DECLARE @Max INT
SELECT @Max=Max(Valoracion) FROM Visita

SELECT Nickname, COALESCE(Comentario, ' ') as Comentarios, Valoracion, Fecha FROM Visita
WHERE Valoracion=@Max

## 22
Cuántos usuarios hay registrados por ciudad

In [74]:
SELECT C.Ciudad, COUNT(*) as [Total Usuarios] FROM Usuario U
INNER JOIN Ciudad C ON U.ID_Ciudad=C.ID_Ciudad
GROUP BY C.Ciudad

## 23
Cuáles son los comentarios que tiene Taipak MonteCarlo

In [75]:
SELECT COALESCE(V.Comentario,'') as Comentarios FROM Visita V
INNER JOIN Lugar L ON V.ID_Lugar=L.ID_Lugar
WHERE L.Nombre='Taipak MonteCarlo'

## 24
Cuántas visitas hay registradas en **MUSIK**

In [76]:
SELECT COUNT(*) AS Visitas FROM Visita V
INNER JOIN Lugar L ON V.ID_Lugar=L.ID_Lugar
WHERE L.Nombre='MUSIK'

## 25
Se desea saber si hay logros que no han sido conseguidos

In [77]:
WITH Conseguidos (Nombre, ID, Total)
AS(
    SELECT L.Nombre, L.ID_Logro, Count(*) FROM Logro_Usuario LU
    INNER JOIN Logro L ON LU.ID_Logro=L.ID_Logro
    --WHERE L.ID_Logro
    GROUP BY L.Nombre, L.ID_Logro
)
SELECT L.Nombre, L.ID_Logro FROM Logro_Usuario LU
FULL JOIN Logro L ON LU.ID_Logro=L.ID_Logro
WHERE L.ID_Logro NOT IN (Select ID FROM Conseguidos)
GROUP BY L.Nombre, L.ID_Logro

## 26
Quién es el usuario que más logros ha conseguido

In [78]:
SELECT TOP 1 U.Nickname, COUNT(*) as [Logros conseguidos] FROM Logro_Usuario L
INNER JOIN Usuario U ON L.Nickname=U.Nickname
GROUP BY U.Nickname
ORDER BY [Logros conseguidos] DESC

## 27
Cuáles son los lugares que han sido visitados en el año actual

In [79]:
SELECT L.Nombre FROM Lugar L
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
WHERE YEAR(V.Fecha)=YEAR(GETDATE())

## 28
Qué lugares de los visitados durante 2019 tuvieron comentarios de __Muy malo__

In [80]:
SELECT L.Nombre FROM Lugar L
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
WHERE Comentario='Muy malo'

## 29
Qué lugares hay registrados en Guadalajara

In [81]:
SELECT L.Nombre FROM Lugar L
INNER JOIN Ciudad C ON L.ID_Ciudad=C.ID_Ciudad
WHERE C.Ciudad='Guadalajara'

## 30
Cuáles son los 3 usuarios más activos

In [82]:
SELECT TOP 3 U.Nickname, COUNT(*) [Total de visitas] FROM Visita V
INNER JOIN Usuario U ON V.Nickname=U.Nickname
GROUP BY U.Nickname
ORDER BY [Total de visitas] DESC

## 31
Cuáles son los 3 intereses menos populares

In [83]:
SELECT TOP 3 I.Interes, count(*) as Cantidad FROM USUARIO U 
INNER JOIN INTERES_USUARIO IU
ON U.Nickname=IU.Nickname 
INNER JOIN INTERES I 
ON IU.ID_Interes=I.ID_Interes 
GROUP BY Interes
ORDER BY Cantidad ASC

## 32
Mostrar la cantidad de visitas que tuvo cada lugar en cada año

In [84]:
WITH [CantVisitas] (ID_Lugar, Fecha, Cantidad)
as 
(
    Select L.ID_Lugar, DATEPART(YY,Fecha), count(*) as Cantidad from VISITA V 
    INNER JOIN LUGAR L 
    ON V.ID_Lugar=L.ID_Lugar 
    GROUP BY L.ID_Lugar, Fecha
)

Select ID_Lugar, COALESCE([2010], ' ') as [2010],COALESCE([2011], ' ') as [2011],COALESCE([2012], ' ') as [2012],COALESCE([2013], ' ') as [2013],
                 COALESCE([2014], ' ') as [2014],COALESCE([2015], ' ') as [2015],COALESCE([2016], ' ') as [2016],COALESCE([2017], ' ') as [2017],
                 COALESCE([2018], ' ') as [2018],COALESCE([2019], ' ') as [2019],COALESCE([2020], ' ') as [2020] from CantVisitas
PIVOT 
(
    SUM(Cantidad)
    FOR Fecha IN ([2010],[2011],[2012],[2013],[2014],[2015],[2016],[2017],[2018],[2019],[2020])
) AS VisitasPorAño

## 33
Mostrar las ultimas 5 visitas que se los usuarios han hecho en distintas ciudades

In [85]:
Select TOP 5 U.Nickname, L.Nombre, V.Fecha from VISITA V 
INNER JOIN USUARIO U 
ON V.Nickname=U.Nickname
INNER JOIN Lugar L 
ON V.ID_Lugar=L.ID_Lugar 
WHERE L.ID_Ciudad<>U.ID_Ciudad
ORDER BY V.Fecha DESC

## 34
Mostrar los usuarios que nunca han salido de la ciudad

In [86]:
Select Nickname from USUARIO
WHERE Nickname not in (SELECT DISTINCT U.Nickname AS Usuario FROM USUARIO U 
                            INNER JOIN VISITA V 
                            ON U.Nickname=V.Nickname
                            INNER JOIN LUGAR L 
                            ON V.ID_Lugar=L.ID_Lugar
                            WHERE U.ID_Ciudad<>L.ID_Ciudad)

## 35
Muestra las 2 ciudades mas populares (las que tienen mas visitas)

In [87]:
SELECT TOP 2 C.Ciudad, COUNT(*) as Visitas FROM VISITA V 
INNER JOIN LUGAR L 
ON V.ID_Lugar=L.ID_LUGAR 
INNER JOIN Ciudad C 
ON L.ID_Ciudad=C.ID_Ciudad
GROUP BY C.Ciudad
ORDER BY Visitas DESC

## 36
Muestra todas las etiquetas que hay en Culiacan

In [88]:
SELECT DISTINCT E.ID_Etiqueta, Nombre_Eti FROM LUGAR L 
INNER JOIN ETIQUETA E 
ON L.ID_Etiqueta=E.ID_Etiqueta 
WHERE L.ID_Ciudad=1

## 37
Muestra todos los logros que hayan sido conseguidos por al menos 1 usuario

In [89]:
SELECT DISTINCT L.ID_Logro, L.Nombre FROM LOGRO L 
INNER JOIN LOGRO_USUARIO LU 
ON L.ID_Logro=LU.ID_Logro 

## 38
Muestra cual es la etiqueta mas comun entre todos los lugares

In [90]:
SELECT TOP 1 E.ID_Etiqueta,  E.Nombre_eti, Count(*) as Cantidad FROM ETIQUETA E 
INNER JOIN LUGAR L 
ON E.ID_Etiqueta=L.ID_Etiqueta
GROUP BY E.ID_Etiqueta, E.Nombre_eti
ORDER BY Cantidad DESC

## 39
Muestra en que año se hicieron mas visitas

In [91]:
SELECT TOP 1 DATEPART(YY, Fecha) as Año, COUNT(*) as [Cantidad de Visitas] FROM VISITA
GROUP BY DATEPART(YY, Fecha)
ORDER BY [Cantidad de Visitas] DESC

## 40
Muestra cuales son los 2 intereses mas comunes entre los usuarios con una edad entre 18 y 30 años

In [92]:
SELECT TOP 2 I.Interes, COUNT(*) as Cantidad  FROM USUARIO U 
INNER JOIN INTERES_USUARIO IU 
ON U.Nickname=IU.Nickname
INNER JOIN INTERES I 
ON IU.ID_interes=I.ID_interes
WHERE DATEDIFF(YEAR,U.Fecha_Nacimiento,GETDATE())>=18 and DATEDIFF(YEAR,U.Fecha_Nacimiento,GETDATE())<=30
GROUP BY I.Interes 
ORDER BY Cantidad DESC

## 41
Mostrar cuáles son los lugares con la etiqueta de **Música** o **Peliculas**

In [93]:
SELECT L.Nombre, E.Nombre_Eti as Etiqueta FROM Lugar L
INNER JOIN Etiqueta E ON L.ID_Etiqueta=E.ID_Etiqueta
WHERE E.Nombre_Eti='Música' OR E.Nombre_Eti='Peliculas'

## 42
Mostrar los lugares con la categoría **Tienda Departamental**

In [94]:
SELECT L.Nombre FROM Lugar L
INNER JOIN Categoria C ON L.ID_Categoria=C.ID_Categoria
WHERE C.Descripcion='Tienda Departamental'

## 43
Qué lugares se visitaron durante diciembre del 2019

In [95]:
SELECT L.Nombre FROM Lugar L
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
WHERE YEAR(V.Fecha)=2019 AND MONTH(V.Fecha)=12

## 44
Quiénes son los usuarios que están interesados en el coleccionismo

In [96]:
SELECT U.Nickname FROM Usuario U
INNER JOIN Interes_Usuario IU ON U.Nickname=IU.Nickname
INNER JOIN Interes I ON IU.ID_Interes=I.ID_Interes
WHERE I.Interes='Coleccionismo'

## 45
Qué lugares y en qué fecha fueron visitados los lugares con la categoría de **Estadios**

In [97]:
SELECT L.Nombre, V.Fecha FROM Visita V
INNER JOIN Lugar L ON L.ID_Lugar=V.ID_Lugar
INNER JOIN Categoria C ON L.ID_Categoria=C.ID_Categoria
WHERE C.Descripcion='Estadios'
ORDER BY Fecha

## 46
Cuál es el lugar que es menos frecuentado por los hombres

In [98]:
WITH Frecuencia(Nombre, Visitas )
AS 
(
    SELECT L.Nombre, count(*) AS Visitas FROM LUGAR L 
    INNER JOIN VISITA V ON L.ID_Lugar=V.ID_Lugar
    INNER JOIN USUARIO U ON V.Nickname=U.Nickname
    WHERE U.Sexo='M'
    GROUP BY L.Nombre
)
SELECT TOP 1 * FROM Frecuencia
ORDER BY Visitas ASC

## 47
Mostrar a los usuarios que han hecho check-in en **KFC Insurgentes**

In [99]:
SELECT U.Nickname FROM Usuario U
INNER JOIN Visita V ON U.Nickname=V.Nickname
INNER JOIN Lugar L ON V.ID_Lugar=L.ID_Lugar
WHERE L.Nombre='KFC Insurgentes'

## 48
Cuáles son los lugares cuya categoría sea **Tienda de Ropa** y fueron visitados en 2018

In [100]:
SELECT L.Nombre, Fecha FROM Lugar L
INNER JOIN Categoria C ON L.ID_Categoria=C.ID_Categoria
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
WHERE C.Descripcion='Tienda de Ropa' AND YEAR(V.Fecha)=2018

## 49
Mostrar a los usuarios que han hecho check-in en **Sams Club 3 Rios** o en **Panamá Insurgentes** o en **Banbajío**; además de las veces que lo han hecho

In [101]:
SELECT U.Nickname as Usuario, L.Nombre, COUNT(*) as [Número de visitas] FROM Visita V
INNER JOIN Usuario U ON U.Nickname=V.Nickname
INNER JOIN Lugar L ON V.ID_Lugar=L.ID_Lugar
WHERE L.Nombre='Sams Club 3 Rios' OR L.Nombre='Panamá Insurgentes' OR L.Nombre='Banbajío'
GROUP BY U.Nickname, L.Nombre
ORDER BY Usuario, L.Nombre, [Número de visitas] ASC

## 50
Mostrar los lugares asociados a restaurantes y que se realizaron check-in en 2019

In [102]:
SELECT L.Nombre, Fecha FROM Lugar L
INNER JOIN Categoria C ON L.ID_Categoria=C.ID_Categoria
INNER JOIN Visita V ON L.ID_Lugar=V.ID_Lugar
WHERE C.Descripcion='Restaurantes' AND YEAR(Fecha)=2019